In [ ]:
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing import image as ksimage
from keras.callbacks import ModelCheckpoint
from keras.metrics import categorical_accuracy
from keras.models import Sequential, load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import LeakyReLU, Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import Activation, Dropout
import warnings; warnings.filterwarnings('ignore')

best_model_path="best.keras"

## preprocessing functions

In [ ]:
def ohe(x):
    return OneHotEncoder(categories='auto')\
           .fit(x.reshape(-1,1)).transform(x.reshape(-1,1))\
           .toarray().astype('int64')
def tts(X,y):
    x_train,x_test,y_train,y_test=\
    train_test_split(X,y,test_size=.2,random_state=1)
    n=int(len(x_test)/2)
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    return x_train,x_valid,x_test,y_train,y_valid,y_test

In [ ]:
f=h5py.File('./LetterColorImages_123.h5','r')

In [ ]:
letters='абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
backgrounds=np.array(f[list(f.keys())[0]])
labels=np.array(f[list(f.keys())[2]])
images=np.array(f[list(f.keys())[1]])/255

In [ ]:
clabels=ohe(labels)

In [ ]:
x_train1,x_valid1,x_test1,\
y_train1,y_valid1,y_test1=tts(images,clabels)

## model code

In [ ]:
model=Sequential()

model.add(Conv2D(8, (5,5), padding='same'))
model.add(LeakyReLU(alpha=.025))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.1))

model.add(Conv2D(16, (5,5), padding='same'))
model.add(LeakyReLU(alpha=.025))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.1))

model.add(Conv2D(32, (5,5), padding='same'))
model.add(LeakyReLU(alpha=.025))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.1))

model.add(GlobalMaxPooling2D())
model.add(Dense(128))
model.add(Dense(33))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=[categorical_accuracy])

# Train

In [ ]:
checkpointer=ModelCheckpoint(filepath=best_model_path,verbose=2,save_best_only=True)
model.fit(x_train1, y_train1, epochs=200, batch_size=64, verbose=2, validation_data=(x_valid1,y_valid1), callbacks=[checkpointer])

Epoch 1/200

Epoch 1: val_loss improved from inf to 3.47053, saving model to best.keras
178/178 - 18s - 101ms/step - categorical_accuracy: 0.0341 - loss: 3.4870 - val_categorical_accuracy: 0.0345 - val_loss: 3.4705
Epoch 2/200

Epoch 2: val_loss improved from 3.47053 to 3.37500, saving model to best.keras
178/178 - 20s - 115ms/step - categorical_accuracy: 0.0507 - loss: 3.3922 - val_categorical_accuracy: 0.0620 - val_loss: 3.3750
Epoch 3/200

Epoch 3: val_loss improved from 3.37500 to 3.30929, saving model to best.keras
178/178 - 20s - 113ms/step - categorical_accuracy: 0.0704 - loss: 3.2739 - val_categorical_accuracy: 0.0627 - val_loss: 3.3093
Epoch 4/200

Epoch 4: val_loss improved from 3.30929 to 3.08482, saving model to best.keras
178/178 - 17s - 96ms/step - categorical_accuracy: 0.0964 - loss: 3.1370 - val_categorical_accuracy: 0.1191 - val_loss: 3.0848
Epoch 5/200

Epoch 5: val_loss improved from 3.08482 to 2.80644, saving model to best.keras
178/178 - 16s - 90ms/step - categoric

train-accuracy: 0.9096

valid-accuracy: 0.9056

# Evaluate model

In [ ]:
model.load_weights(best_model_path)
model.evaluate(x_test1,y_test1)

45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - categorical_accuracy: 0.8843 - loss: 0.3937


[0.3847358226776123, 0.8900634050369263]

 test-accuracy: 0.8843